In [121]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk import bigrams
from nltk import word_tokenize
from sklearn.linear_model import LogisticRegression



In [144]:
## change to the appropriate path
df = pd.read_csv('structured_data.csv',encoding='latin1',sep='\t') # seperator is \t as file is tab-delimited
df.head()

,Unnamed: 0,report status,intro text,Date of inspection visit:,Date of publication:,Inspection report,Summary of findings,Is the service safe?,Is the service effective?,Is the service caring?,...,Is the service safe?.1,Our findings,Is the service effective?.1,Our findings.1,Is the service caring?.1,Our findings.2,Is the service responsive?.1,Our findings.3,Is the service well-led?.1,Our findings.4
0,0,outstanding,Embrace (Geffen) Limited Nunthorpe Oaks Inspe...,21 February 2017 07 March 2017,24 May 2017 Outstanding Good Good ...,24 May 2017,Overall summary We inspected Nunthorpe Oaks...,The service was safe. People told us they f...,The service was effective. Staff had the k...,This service was extremely caring People w...,...,Good,People and their relatives told us they fel...,Good,People who used the service told us staff p...,Outstanding,"People, relatives and professionals praised...",Outstanding,People who used the service and relatives c...,Outstanding,The service was exceptionally well-led. It ...
1,1,outstanding,Insight Specialist Behavioural Service Ltd Asp...,04 December 2017,09 February 2018 Outstanding Good ...,09 February 2018,Overall summary Aspley House is a residenti...,The service remains Good,The service remains good.,The service was exceptionally caring. The p...,...,Good,The safety of everyone at the service both ...,Good,"Health professionals told us, ""I am always ...",Outstanding,A health and social care professional told ...,Outstanding,"People lived active and meaningful lives, t...",Outstanding,"Staff told us, ""The philosophy of the provi..."
2,2,outstanding,Miss S G Howard Victoria Lodge Care Home,26 January 2017,22 March 2017 Outstanding Good Goo...,11 Victoria Road Acocks Green Birmingham Wes...,Overall summary This unannounced inspection...,The service was safe People told us they fe...,The service was effective. There were arra...,The service was very caring. People receive...,...,Good,People and relatives all gave us very posit...,Good,People and relatives we spoke with told us ...,Outstanding,People living in Victoria Lodge were happy ...,Good,Relatives told us that the supportive and r...,Outstanding,The registered manager and provider promote...
3,3,outstanding,Hale Place Care Homes Limited Hale Place Farmh...,04 September 2017 07 September 2017,10 November 2017 Outstanding Good ...,79 Old Road East Peckham Tonbridge Kent TN12...,Overall summary This inspection took place ...,Hale Place Farmhouse was safe. People were ...,Hale Place Farmhouse was effective. Staff r...,Outstanding The service was outstanding i...,...,Good,People told us that they felt safe living a...,Good,People and their relatives spoke positively...,Outstanding,People and their relatives were consistentl...,Outstanding,People were receiving a person centred serv...,Outstanding,People and their relatives were consistentl...
4,4,outstanding,Healthcare Homes (LSC) Limited Sandown Park Ca...,09 January 2018 10 January 2018,27 February 2018 Outstanding Good ...,27 February 2018,Overall summary Our inspection took place o...,The service was safe. Effective systems wer...,"The service was effective. People's likes, ...",Outstanding 4 Sandown Park Care Home Insp...,...,Good,People and relatives provided positive feed...,Good,People and relatives told us they felt staf...,Outstanding,The service continued to provide outstandin...,Outstanding,The service continued to provide outstandin...,Outstanding,The service's management and care team were...


In [190]:
#define parameters of count vectorizer
vec = CountVectorizer(analyzer="word",stop_words='english',
                                   ngram_range=(1, 2),
                                   tokenizer=word_tokenize,
                                   max_features=10000)


lr=LogisticRegression()


#define a scikit learn pipeline
from sklearn.pipeline import Pipeline
bigram_clf = Pipeline([


    ('vectorizer', vec),
    
    ('classifier', lr)     # or any other classifier deemed useful!
   
])










In [191]:
ordered_status = ['good','inadequate','outstanding', 'requires_improvement' ]

In [192]:
#create a label that can be processed by the ML algorithm.

from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
df['label'] = lb_make.fit_transform(df['report status'])

#check original column and label column

df[['report status', 'label']]




,report status,label
0,outstanding,2
1,outstanding,2
2,outstanding,2
3,outstanding,2
4,outstanding,2
5,requires_improvement,3
6,requires_improvement,3
7,requires_improvement,3
8,requires_improvement,3
9,requires_improvement,3


In [193]:



#define X as text from column 'Our findings'
X=df['Our findings'].values
#define y as text from column 'label' created above
y=df.label.values





In [194]:
# split data into train and test set randomly (but in a way that is repeatable with seed random_state)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [195]:
# fit the train data into our transformer/classifier pipeline

print(bigram_clf.fit(X_train, y_train))



Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
 ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])


In [196]:
#score the test data

print(bigram_clf.score(X_test, y_test))

0.75


### debug classifiers and explain their predictions

In [197]:
import eli5

In [201]:
eli5.show_weights(lr, top=20,vec=vec,target_names=ordered_status) 

In [221]:
#explain prediction of a particular text
eli5.explain_prediction(lr, X_test[1],vec=vec, top=15,target_names=ordered_status)

Explanation(estimator="LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,\n          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,\n          verbose=0, warm_start=False)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='good', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='staff', weight=0.7673072250269892, std=None, value=19.0), FeatureWeight(feature='home', weight=0.6995796746929905, std=None, value=10.0), FeatureWeight(feature='taken', weight=0.371093186235298, std=None, value=13.0), FeatureWeight(feature='people', weight=0.2781433835812618, std=None, value=22.0), FeatureWeight(feature='risks', weight=0.27433050351195665, std=None, value=8.0), FeatureWeight(feature='concerns', weight=0.1776958972251079, std=None, value=4.0)], neg=[FeatureWeight(feature='.', weight=-0.939617610026045, std=None, value=63.0), FeatureWeight(feature="'s", weight=-0.8838867957449301, std=None, value=10.0), FeatureWeight(feature='inspection', weight=-0.7529787499223126, std=None, value=15.0), FeatureWeight(feature='registered', weight=-0.4336702844870125, std=None, value=7.0), FeatureWeight(feature='meant', weight=-0.2782095898244476, std=None, value=8.0), FeatureWeight(feature="people 's", weight=-0.27466915007976106, std=None, value=6.0), FeatureWeight(feature='care', weight=-0.22765078301084124, std=None, value=4.0), FeatureWeight(feature='registered manager', weight=-0.22384443781703903, std=None, value=7.0), FeatureWeight(feature='2017', weight=-0.21825758365264128, std=None, value=10.0)], pos_remaining=167, neg_remaining=354), proba=0.019994758970701337, score=-4.227173812096503, weighted_spans=None), TargetExplanation(target='inadequate', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='risk', weight=1.3214294003356275, std=None, value=26.0), FeatureWeight(feature='inspection', weight=0.5725722476623044, std=None, value=15.0), FeatureWeight(feature='people', weight=0.4197515267911069, std=None, value=22.0), FeatureWeight(feature='did', weight=0.38817808778959767, std=None, value=8.0), FeatureWeight(feature='person', weight=0.3495098958216677, std=None, value=22.0), FeatureWeight(feature="'s", weight=0.29310834334067026, std=None, value=10.0), FeatureWeight(feature='2017', weight=0.1865201840515977, std=None, value=10.0), FeatureWeight(feature='medicine', weight=0.1759527081902137, std=None, value=5.0)], neg=[FeatureWeight(feature='.', weight=-3.748135872694793, std=None, value=63.0), FeatureWeight(feature=',', weight=-0.4900493242277408, std=None, value=29.0), FeatureWeight(feature='home', weight=-0.39689773522784166, std=None, value=10.0), FeatureWeight(feature='place', weight=-0.3895040031967978, std=None, value=11.0), FeatureWeight(feature='staff', weight=-0.3760603912043873, std=None, value=19.0), FeatureWeight(feature='saw', weight=-0.35805987844766246, std=None, value=8.0), FeatureWeight(feature="''", weight=-0.16954628881055922, std=None, value=4.0)], pos_remaining=260, neg_remaining=261), proba=0.7853500950525718, score=0.26131196602465534, weighted_spans=None), TargetExplanation(target='outstanding', feature_weights=FeatureWeights(pos=[FeatureWeight(feature="'s", weight=0.2679589741056549, std=None, value=10.0), FeatureWeight(feature='service', weight=0.243019813150383, std=None, value=3.0), FeatureWeight(feature="''", weight=0.18076391413308784, std=None, value=4.0), FeatureWeight(feature='actions', weight=0.17751081586553222, std=None, value=6.0), FeatureWeight(feature='.', weight=0.15172440965676562, std=None, value=63.0), FeatureWeight(feature=', ``', weight=0.14816268003365504, std=None, value=4.0), FeatureWeight(feature=". ''", weight=0.14513799904335234, std=None, value=4.0)], neg=[FeatureWeight(feature=',', weight=-2.196048424408043, std=None, value=29.0), FeatureWeight(feature='risk', weight=-0.8722244442298677, std=None, 

In [228]:
#explaining the predictions of  ad hoc text. here a (kind of nonsensical) sentence based on tokens 
# more important in the 'outstanding' class
# change it to see how the classifier reacts.

eli5.explain_prediction(lr, "a person told us that having service safety actions rated as great",
                        vec=vec, top=15,target_names=ordered_status)

Explanation(estimator="LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,\n          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,\n          verbose=0, warm_start=False)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='good', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='told', weight=0.026396875185057324, std=None, value=1.0), FeatureWeight(feature='safety', weight=0.024693568817940045, std=None, value=1.0)], neg=[FeatureWeight(feature='service', weight=-0.04201325351257713, std=None, value=1.0), FeatureWeight(feature='actions', weight=-0.02033315469444714, std=None, value=1.0), FeatureWeight(feature='person told', weight=-0.01978981182022398, std=None, value=1.0), FeatureWeight(feature='person', weight=-0.007890366607395869, std=None, value=1.0), FeatureWeight(feature='having', weight=-0.004299378764930685, std=None, value=1.0), FeatureWeight(feature='<BIAS>', weight=-0.001199380114288144, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.24703728134723935, score=-0.04443490151086558, weighted_spans=None), TargetExplanation(target='inadequate', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='service', weight=0.041264681677007545, std=None, value=1.0), FeatureWeight(feature='person', weight=0.01588681344643944, std=None, value=1.0), FeatureWeight(feature='having', weight=0.00037417462726210994, std=None, value=1.0)], neg=[FeatureWeight(feature='told', weight=-0.026962528840450334, std=None, value=1.0), FeatureWeight(feature='safety', weight=-0.019815437444132136, std=None, value=1.0), FeatureWeight(feature='person told', weight=-0.018337801268965868, std=None, value=1.0), FeatureWeight(feature='actions', weight=-0.00590244481169124, std=None, value=1.0), FeatureWeight(feature='<BIAS>', weight=-0.002775795088813512, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.25059453862818026, score=-0.016268337703343995, weighted_spans=None), TargetExplanation(target='outstanding', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='service', weight=0.08100660438346101, std=None, value=1.0), FeatureWeight(feature='person told', weight=0.04318349129745468, std=None, value=1.0), FeatureWeight(feature='safety', weight=0.0347000666552695, std=None, value=1.0), FeatureWeight(feature='actions', weight=0.029585135977588703, std=None, value=1.0), FeatureWeight(feature='told', weight=0.021970895716039947, std=None, value=1.0), FeatureWeight(feature='<BIAS>', weight=0.004094654668373436, std=None, value=1.0)], neg=[FeatureWeight(feature='person', weight=-0.012175218822749113, std=None, value=1.0), FeatureWeight(feature='having', weight=-0.0027048751090137995, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.2777882353700545, score=0.19966075476642436, weighted_spans=None), TargetExplanation(target='requires_improvement', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='having', weight=0.008653131779409485, std=None, value=1.0)], neg=[FeatureWeight(feature='service', weight=-0.0968982665735267, std=None, value=1.0), FeatureWeight(feature='safety', weight=-0.055438676015407924, std=None, value=1.0), FeatureWeight(feature='told', weight=-0.03379533029370691, std=None, value=1.0), FeatureWeight(feature='person told', weight=-0.01673020005952938, std=None, value=1.0), FeatureWeight(feature='actions', weight=-0.012644695061726205, std=None, value=1.0), FeatureWeight(feature='person', weight=-0.012482146786454654, std=None, value=1.0), FeatureWeight(feature='<BIAS>', weight=-0.00378706334464608, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.224579944654526, score=-0.22312324635558836, weighted_spans=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None)